# Master the Most Time-consuming Task in DS/ML, Part I
## Dealing with common data problems
<img src='images/clean2.jpg'></img>
<figcaption style="text-align: center;">
    <strong>
        Photo by 
        <a href='https://unsplash.com/@thecreative_exchange?utm_source=medium&utm_medium=referral'>Steve Johnson</a>
        on 
        <a href='https://unsplash.com/?utm_source=medium&utm_medium=referral'>Unsplash</a>
    </strong>
</figcaption>

### Introduction <small id='intro'></small>

### Setup <small id='setup'></small>

In [ ]:
# Load necessary libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

### Data Type Constraints <small id='type'></small>

### Data Range Constraints <small id='range'></small>

### Uniqueness Constraints <small id='unique'></small>